In [ ]:
# Add anna to the path
import os
import sys
module_path = os.path.abspath(os.path.join('../../../anna'))
if module_path not in sys.path:
    sys.path.append(module_path)

DATA_DIR = "../../../data"

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import anna.data.dataset.reuters21578 as reuters
import anna.model.premade as models
import anna.summary.extract as summary

tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

# Colors for Plots
colors = [(31, 119, 180),  (174, 199, 232), (255, 127, 14),  (255, 187, 120),
          (44, 160, 44),   (152, 223, 138), (214, 39, 40),   (255, 152, 150),
          (148, 103, 189), (197, 176, 213), (140, 86, 75),   (196, 156, 148),
          (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
          (188, 189, 34),  (219, 219, 141), (23, 190, 207),  (158, 218, 229)]

for i in range(len(colors)):    
    r, g, b = colors[i]    
    colors[i] = (r / 255., g / 255., b / 255.)

In [ ]:
# Load data
train_docs, test_docs, unused_docs, labels = reuters.fetch_and_parse(DATA_DIR)

# Standard Reuters Config
val_size = 777
epochs = 20
shuffle = 10000
eval_every = 100

In [ ]:
# Create MLP classifier
for layers in range(4):
    for size in [512, 1024, 2048, 4096]:
        model = models.AVGxBR(DATA_DIR, labels, layers=layers, hidden_size=size,
                              name="avg_br_{}_{}".format(layers, size),
                              folder_name="model-sizes")

        # Train and evaluate
        print("Model: {}, Layers: {}, Size: {}".format(model, layers, size))
        model.train(train_docs, test_docs,
                    val_size=val_size, epochs=epochs, shuffle=shuffle, eval_every=eval_every)

    # Delete to save memory
    del model

In [ ]:
metrics = summary.parse_all(os.path.join(DATA_DIR, "model-sizes"))

In [ ]:
num_epochs = min([len(m["val"]["perf/accuracy"]) for m in metrics.values()])
remove_first = 0

x = range(num_epochs)[remove_first:]
for metric_name in ["perf/accuracy", "perf/ebF1"]:
    pre, (ax1, ax2) = plt.subplots(1, 2, figsize=[15, 7])


    for train_set, ax in [("train", ax1), ("val", ax2)]:
        ax.set_xlabel("epoch")
        ax.set_ylabel("{}/{}".format(train_set, metric_name[len("perf/"):]))
        ax.spines["top"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)

        for name, metric in metrics.items():
            y = metric[train_set][metric_name][remove_first:num_epochs]
            y = [m[1] for m in y]
            drop = float(name[len("avg_br_"):])
            color = str(1. - min([drop * 2 + 0.05, 1.0]))
            color = colors[1] if drop == .4 else color
            color = colors[3] if drop == .5 else color
            ax.plot(x, y, color=color, label="rate {}".format(drop))

        ax.legend(loc=4)